# Chapter 29: Timezones and ZoneInfo

This notebook covers timezone handling in Python using `timezone`, `ZoneInfo`, and the distinction between naive and aware datetimes. Correct timezone handling is critical for applications that operate across regions.

## Key Concepts
- **Naive datetimes**: No timezone information attached
- **Aware datetimes**: Include timezone information via `tzinfo`
- **`timezone.utc`**: The built-in UTC timezone constant
- **`ZoneInfo`**: Access to the IANA timezone database (Python 3.9+)
- **`astimezone()`**: Convert between timezones

## Section 1: Naive vs Aware Datetimes

A **naive** datetime has no timezone info (`tzinfo is None`). An **aware** datetime carries explicit timezone information. Mixing them causes errors, so it is important to be consistent.

In [ ]:
from datetime import datetime, timezone

# Naive datetime -- no timezone
naive: datetime = datetime(2025, 6, 15, 12, 0)
print(f"Naive datetime: {naive}")
print(f"tzinfo:         {naive.tzinfo}")
print(f"Is naive:       {naive.tzinfo is None}")

# Aware datetime -- with timezone
aware: datetime = datetime(2025, 6, 15, 12, 0, tzinfo=timezone.utc)
print(f"\nAware datetime: {aware}")
print(f"tzinfo:         {aware.tzinfo}")
print(f"Is aware:       {aware.tzinfo is not None}")

In [ ]:
from datetime import datetime, timezone

# Cannot compare naive and aware datetimes
naive: datetime = datetime(2025, 6, 15, 12, 0)
aware: datetime = datetime(2025, 6, 15, 12, 0, tzinfo=timezone.utc)

try:
    result = naive < aware
except TypeError as e:
    print(f"Comparison error: {e}")

# Cannot subtract naive from aware
try:
    diff = aware - naive
except TypeError as e:
    print(f"Subtraction error: {e}")

print("\nLesson: never mix naive and aware datetimes.")

## Section 2: Working with UTC

UTC (Coordinated Universal Time) is the standard reference timezone. Always store timestamps in UTC and convert to local time only for display.

In [ ]:
from datetime import datetime, timezone

# Create a UTC datetime
utc_now: datetime = datetime.now(timezone.utc)
print(f"UTC now: {utc_now}")
print(f"tzname:  {utc_now.tzname()}")

# Construct a specific UTC datetime
dt: datetime = datetime(2025, 6, 15, 12, 0, tzinfo=timezone.utc)
print(f"\nSpecific UTC: {dt}")
print(f"tzinfo == timezone.utc: {dt.tzinfo == timezone.utc}")
print(f"tzname: {dt.tzname()}")

In [ ]:
from datetime import datetime, timezone, timedelta

# Fixed-offset timezones using timezone(timedelta(...))
utc_plus_5: timezone = timezone(timedelta(hours=5))
utc_minus_8: timezone = timezone(timedelta(hours=-8))

dt_plus5: datetime = datetime(2025, 6, 15, 17, 0, tzinfo=utc_plus_5)
dt_minus8: datetime = datetime(2025, 6, 15, 4, 0, tzinfo=utc_minus_8)

print(f"UTC+5:  {dt_plus5}")
print(f"UTC-8:  {dt_minus8}")

# Both represent the same instant in time (12:00 UTC)
print(f"\nSame instant? {dt_plus5 == dt_minus8}")
print(f"Plus5 in UTC:  {dt_plus5.astimezone(timezone.utc)}")
print(f"Minus8 in UTC: {dt_minus8.astimezone(timezone.utc)}")

## Section 3: ZoneInfo -- IANA Timezones

The `zoneinfo` module (Python 3.9+) provides access to the IANA timezone database. This gives you named timezones like `"America/New_York"` that automatically handle daylight saving time (DST).

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

# Create a datetime in a specific timezone
eastern: ZoneInfo = ZoneInfo("America/New_York")
dt: datetime = datetime(2025, 6, 15, 12, 0, tzinfo=eastern)

print(f"New York:    {dt}")
print(f"tzinfo:      {dt.tzinfo}")
print(f"tzname:      {dt.tzname()}")
print(f"UTC offset:  {dt.utcoffset()}")

# Some common timezones
zones: list[str] = [
    "America/New_York",
    "America/Chicago",
    "America/Los_Angeles",
    "Europe/London",
    "Europe/Berlin",
    "Asia/Tokyo",
]
print("\nCommon timezones:")
for zone_name in zones:
    tz: ZoneInfo = ZoneInfo(zone_name)
    local_dt: datetime = datetime(2025, 6, 15, 12, 0, tzinfo=tz)
    print(f"  {zone_name:25s} UTC offset: {local_dt.utcoffset()}")

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo, available_timezones

# List available timezones (there are hundreds)
all_zones: set[str] = available_timezones()
print(f"Total available timezones: {len(all_zones)}")

# Show a sample of US timezones
us_zones: list[str] = sorted(tz for tz in all_zones if tz.startswith("US/"))
print(f"\nUS timezones:")
for tz_name in us_zones:
    print(f"  {tz_name}")

## Section 4: Timezone Conversion with `astimezone()`

The `astimezone()` method converts an aware datetime to a different timezone. The underlying instant in time stays the same -- only the representation changes.

In [ ]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

# Start with a UTC time
utc_dt: datetime = datetime(2025, 6, 15, 12, 0, tzinfo=timezone.utc)
print(f"UTC:        {utc_dt}")

# Convert to Eastern Time (UTC-4 in June due to EDT)
eastern: datetime = utc_dt.astimezone(ZoneInfo("America/New_York"))
print(f"New York:   {eastern}")
print(f"Hour:       {eastern.hour}")  # 8 (12 - 4)
print(f"Same date:  {eastern.date() == utc_dt.date()}")

# Convert to other timezones
tokyo: datetime = utc_dt.astimezone(ZoneInfo("Asia/Tokyo"))
london: datetime = utc_dt.astimezone(ZoneInfo("Europe/London"))
print(f"\nTokyo:      {tokyo}")
print(f"London:     {london}")

# All represent the same instant
print(f"\nAll equal?  {utc_dt == eastern == tokyo == london}")

In [ ]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

# Practical: display a meeting time across multiple timezones
def show_meeting_times(
    meeting_utc: datetime,
    timezones: list[str],
) -> None:
    """Display a UTC meeting time in multiple timezones."""
    print(f"Meeting time (UTC): {meeting_utc.strftime('%Y-%m-%d %H:%M %Z')}")
    print("-" * 50)
    for tz_name in timezones:
        tz: ZoneInfo = ZoneInfo(tz_name)
        local: datetime = meeting_utc.astimezone(tz)
        print(f"  {tz_name:25s} {local.strftime('%H:%M %Z (%Y-%m-%d)')}")

meeting: datetime = datetime(2025, 6, 15, 15, 0, tzinfo=timezone.utc)
offices: list[str] = [
    "America/New_York",
    "America/Los_Angeles",
    "Europe/London",
    "Europe/Berlin",
    "Asia/Tokyo",
    "Australia/Sydney",
]
show_meeting_times(meeting, offices)

## Section 5: Daylight Saving Time

`ZoneInfo` automatically handles DST transitions. The same timezone can have different UTC offsets depending on the date.

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo

eastern: ZoneInfo = ZoneInfo("America/New_York")

# Summer (EDT -- Eastern Daylight Time, UTC-4)
summer: datetime = datetime(2025, 6, 15, 12, 0, tzinfo=eastern)
print(f"Summer: {summer}")
print(f"  tzname:     {summer.tzname()}")
print(f"  UTC offset: {summer.utcoffset()}")
print(f"  DST offset: {summer.dst()}")

# Winter (EST -- Eastern Standard Time, UTC-5)
winter: datetime = datetime(2025, 12, 15, 12, 0, tzinfo=eastern)
print(f"\nWinter: {winter}")
print(f"  tzname:     {winter.tzname()}")
print(f"  UTC offset: {winter.utcoffset()}")
print(f"  DST offset: {winter.dst()}")

# The offset changes depending on date
print(f"\nSummer offset != Winter offset: {summer.utcoffset() != winter.utcoffset()}")

## Section 6: Making Naive Datetimes Aware

You can attach timezone info to a naive datetime using `replace()` or by passing `tzinfo` at construction time. Use `replace()` when you know the naive time is already in a specific timezone.

In [ ]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

# A naive datetime (no timezone)
naive: datetime = datetime(2025, 6, 15, 14, 30)
print(f"Naive: {naive} (tzinfo={naive.tzinfo})")

# Attach UTC timezone using replace
# This says "this time IS in UTC" -- it does not convert
as_utc: datetime = naive.replace(tzinfo=timezone.utc)
print(f"As UTC: {as_utc}")

# Attach a named timezone
as_eastern: datetime = naive.replace(tzinfo=ZoneInfo("America/New_York"))
print(f"As Eastern: {as_eastern}")

# These are different instants in time!
print(f"\nAs UTC in UTC:     {as_utc.astimezone(timezone.utc)}")
print(f"As Eastern in UTC: {as_eastern.astimezone(timezone.utc)}")
print(f"Same instant? {as_utc == as_eastern}")

## Section 7: Best Practices for Timezone Handling

Follow these guidelines to avoid common timezone bugs.

In [ ]:
from datetime import datetime, timezone
from zoneinfo import ZoneInfo

# BEST PRACTICE 1: Always use aware datetimes
# Bad:  datetime.now()        -- returns naive local time
# Good: datetime.now(tz=...)  -- returns aware time
now_utc: datetime = datetime.now(tz=timezone.utc)
now_eastern: datetime = datetime.now(tz=ZoneInfo("America/New_York"))

print(f"UTC now:     {now_utc}")
print(f"Eastern now: {now_eastern}")

# BEST PRACTICE 2: Store in UTC, display in local
stored_event: datetime = datetime(2025, 6, 15, 18, 0, tzinfo=timezone.utc)
user_tz: ZoneInfo = ZoneInfo("Asia/Tokyo")
displayed: datetime = stored_event.astimezone(user_tz)
print(f"\nStored (UTC): {stored_event}")
print(f"Display (Tokyo): {displayed}")

# BEST PRACTICE 3: Use ZoneInfo, not fixed offsets
# Fixed offsets do not handle DST correctly
print("\nZoneInfo handles DST automatically:")
tz: ZoneInfo = ZoneInfo("America/New_York")
jun: datetime = datetime(2025, 6, 15, tzinfo=tz)
dec: datetime = datetime(2025, 12, 15, tzinfo=tz)
print(f"  June offset:     {jun.utcoffset()}")
print(f"  December offset: {dec.utcoffset()}")

## Summary

### Naive vs Aware
- **Naive**: `datetime.tzinfo is None` -- no timezone information
- **Aware**: `datetime.tzinfo is not None` -- explicit timezone
- Never mix naive and aware datetimes in comparisons or arithmetic

### UTC
- Use `timezone.utc` for the built-in UTC timezone
- Use `datetime.now(timezone.utc)` instead of `datetime.utcnow()` (deprecated)
- Store timestamps in UTC; convert to local time for display only

### ZoneInfo (Python 3.9+)
- `ZoneInfo("America/New_York")` gives IANA timezone with DST support
- `available_timezones()` lists all known timezone names
- `astimezone()` converts between timezones, preserving the instant

### Best Practices
1. Always use aware datetimes
2. Store in UTC, display in local time
3. Use `ZoneInfo` instead of fixed UTC offsets for named timezones
4. Use `replace(tzinfo=...)` to attach timezone info to a naive datetime